# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [7]:
for path in song_files[1:]:
    df_new = pd.read_json(path, lines=True)
    df = pd.concat([df,df_new])
    
df.head()   

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
0,ARGSJW91187B9B1D6B,35.21962,North Carolina,-80.01955,JennyAnyKind,218.77506,1,SOQHXMF12AB0182363,Young Boy Blues,0
0,ARKRRTF1187B9984DA,NaN,,NaN,Sonora Santanera,177.47546,1,SOXVLOJ12AB0189215,Amor De Cabaret,0
0,ARKFYS91187B98E58F,NaN,,NaN,Jeff And Sheri Easter,267.70240,1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),0
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[["song_id","title","artist_id","year","duration"]].values.tolist()[0]
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]].values.tolist()[0]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

In [13]:
filepath = log_files[0]

In [14]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [15]:
for path in log_files[1:]:
    df_new = pd.read_json(path, lines=True)
    df = pd.concat([df,df_new])
    
df.head(20)   

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
5,The Fray,Logged In,Sara,F,77,Johnson,209.97179,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Syndicate,200,1542155006796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
6,Chuck Berry,Logged In,Sara,F,78,Johnson,147.06893,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Roll Over Beethoven,200,1542155215796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
7,Dimmu Borgir,Logged In,Sara,F,79,Johnson,468.47955,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Kings Of The Carnival Creation,200,1542155362796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
8,Jimmy Nail,Logged In,Sara,F,80,Johnson,245.65506,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Ain't No Doubt,200,1542155830796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
9,Weezer,Logged In,Sara,F,81,Johnson,189.43955,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Pork And Beans,200,1542156075796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df[df['page'] == "NextSong"]
df['ts'] = pd.to_datetime(df['ts'], unit='ms')

df['hour'] = df['ts'].dt.hour
df['day'] = df['ts'].dt.day
df['week'] = df['ts'].dt.week
df['month'] = df['ts'].dt.month
df['year'] = df['ts'].dt.year
df['weekday'] = df['ts'].dt.weekday

In [17]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,hour,day,week,month,year,weekday
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,...,200,2018-11-14 00:03:22.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,0,14,46,11,2018,2
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,...,200,2018-11-14 00:06:08.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,0,14,46,11,2018,2
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,...,200,2018-11-14 00:11:06.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,0,14,46,11,2018,2
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,...,200,2018-11-14 00:17:37.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,0,14,46,11,2018,2
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,...,200,2018-11-14 00:20:25.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95,0,14,46,11,2018,2


In [18]:
time_data = ["ts", "hour", "day", "week", "month", "year", "weekday"]
column_labels = ["start_time", "hour", "day", "week", "month", "year", "weekday"]

In [19]:
time_df = df[time_data]
time_df.columns = column_labels
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-14 00:03:22.796,0,14,46,11,2018,2
1,2018-11-14 00:06:08.796,0,14,46,11,2018,2
2,2018-11-14 00:11:06.796,0,14,46,11,2018,2
3,2018-11-14 00:17:37.796,0,14,46,11,2018,2
4,2018-11-14 00:20:25.796,0,14,46,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[['userId','firstName','lastName',"gender","level"]]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        print(results)
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.